In [9]:
import os
import requests
from datetime import datetime
import re
import json  # Para exportar el resumen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np


### INVERSION BRUTA FIJA DE LOS GOBIERNOS REGIONALES 
---
Inicia desde RD16709
Termina en RD16735DA 

In [10]:
# ESTA PRIMERA CELDA ES UNA MUESTRA DE LO QUE SE HACE
codigo="RD16735DA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/2000-01/2024-12' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data=response.json() # lo almacenamos en un elemenot llamado data
print(data) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Inversión bruta fija de los gobiernos regionales (millones S/)', 'series': [{'name': 'Inversión bruta fija de los gobiernos regionales (millones S/) - Total (millones de soles)', 'dec': '0'}]}, 'periods': [{'name': '2008', 'values': ['2773.17505119']}, {'name': '2009', 'values': ['4135.09061704']}, {'name': '2010', 'values': ['5043.13886529']}, {'name': '2011', 'values': ['4812.11597454']}, {'name': '2012', 'values': ['6923.01043951']}]}


##### CODIGO DE SERIES

In [11]:
# GENERAMOS LOS CODIGOS
n_serie=np.linspace(9,35,(36-9))
codigos=[]
for elem in n_serie:
    if elem<10:
        numero='0'+str(int(elem))
    else:
        numero=str(int(elem))

    codigo='RD167'+numero+'DA'
    codigos+=[codigo]

print(codigos)


['RD16709DA', 'RD16710DA', 'RD16711DA', 'RD16712DA', 'RD16713DA', 'RD16714DA', 'RD16715DA', 'RD16716DA', 'RD16717DA', 'RD16718DA', 'RD16719DA', 'RD16720DA', 'RD16721DA', 'RD16722DA', 'RD16723DA', 'RD16724DA', 'RD16725DA', 'RD16726DA', 'RD16727DA', 'RD16728DA', 'RD16729DA', 'RD16730DA', 'RD16731DA', 'RD16732DA', 'RD16733DA', 'RD16734DA', 'RD16735DA']


In [12]:
codigo="RD16709DA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/2000-01/2024-12' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data=response.json() # lo almacenamos en un elemenot llamado data
print(data) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Inversión bruta fija de los gobiernos regionales (millones S/)', 'series': [{'name': 'Inversión bruta fija de los gobiernos regionales (millones S/) - Amazonas (millones de soles)', 'dec': '0'}]}, 'periods': [{'name': '2008', 'values': ['107.84795192']}, {'name': '2009', 'values': ['147.6978048']}, {'name': '2010', 'values': ['117.50093039']}, {'name': '2011', 'values': ['142.53619006']}, {'name': '2012', 'values': ['223.89459691']}]}


#### BUCLE PARA EXTRAER LA INFORMACIÓN DE TODOS LOS DICCIONARIOS

In [13]:
inicio="2008" # data disponible solo desde 2008
fin="2024" # termina en 2024 todas las series 
dep_nom=[] # para almacenar el nombre de los departamentos
valores=[] # para almacenar las listas que contendran los datos de los departamentos

fechas = pd.date_range(start=inicio, end=fin, freq='YS') # creamos las fechas para que tengan solo mes y año
fechas_lista = fechas.strftime('%Y').tolist() # ACA ESTAN LAS FECHAS

for codigo in codigos:
    url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/{inicio}/{fin}' 
    response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
    data_1=response.json() # lo almacenamos en un elemento llamado data
    #PRIMERO SE OBTIENEN LOS NOMBRES, que será tmbn el nombre de la columna
    nombre_dep=data_1['config']['series'][0]['name']
    partes = nombre_dep.split(' - ')
    partes_1=partes[1].split(' (')
    nombre_extraido = partes_1[0] # TENER EN MENTE QUE NOMBRE EXTRAIDO SERA EL NOMBRE DE LA COLUMNA
    dep_nom+=[nombre_extraido] # a la lista de nombres que tendremos tmbn como un output
    #SEGUNDO SE OBTIENEN LOS VALORES REGISTRADOS
    series=data_1['periods']
    registros=[elem['values'][0] for elem in series]
    valores+=[registros] # se llena 1 elemento nuevo por cada region
print(dep_nom)
print(len(valores)) # son 25 porque al final contiene los valores totales en formato anual
print(fechas_lista)

['Amazonas', 'Ancash', 'Apurímac', 'Arequipa', 'Ayacucho', 'Cajamarca', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín', 'La Libertad', 'Lambayeque', 'Loreto', 'Madre de Dios', 'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes', 'Ucayali', 'Lima', 'Callao', 'Municipalidad Metropolitana de Lima', 'Total']
27
['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [14]:
columnas = ['Fecha']+dep_nom
#print(len(columnas))
datos=[fechas_lista]+valores
#print(len(datos))

dic_data={}

for i in range(len(datos)): # se arma el diccionario con la data correspondiente a cada columna
    dic_data[columnas[i]]=datos[i]

#print(len(dic_data))

df=pd.DataFrame(dic_data,columns=columnas)
df = df.apply(pd.to_numeric, errors='coerce')
df

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,Huánuco,...,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Lima,Callao,Municipalidad Metropolitana de Lima,Total
0,2008,107.847952,137.191372,80.227281,198.919016,100.180405,71.073810,246.746352,130.278501,85.469185,...,148.282108,91.748756,122.353168,93.954441,53.905085,84.933871,78.262653,67.587287,44.806772,2773.175051
1,2009,147.697805,293.012859,71.664913,307.639191,126.896620,211.054389,305.993902,138.237349,73.862822,...,203.057100,234.615584,222.642133,142.419142,81.176146,151.710934,112.897832,94.368924,139.433823,4135.090617
2,2010,117.500930,541.043618,142.140534,318.432541,183.448783,275.685354,282.272456,182.107426,109.474218,...,330.218633,189.195705,222.326283,125.901346,109.892723,144.365414,149.961559,131.674521,273.139003,5043.138865
3,2011,142.536190,649.156495,112.752968,343.470299,139.331804,460.379504,243.794412,169.243156,193.918775,...,195.761128,116.392387,261.741509,73.029949,135.975018,137.318775,100.426844,119.538740,11.112849,4812.115975
4,2012,223.894597,696.825892,186.290313,370.733660,261.454066,384.987475,608.244709,236.340936,301.055569,...,360.214551,225.451162,357.312459,181.231406,214.718507,129.676773,213.958629,364.763838,77.545928,6923.010440
5,2013,247.529996,361.679792,234.072605,573.170248,360.690289,270.990811,695.597514,225.425188,247.167326,...,421.203015,316.143162,408.790677,199.529286,166.063754,205.076092,252.950041,336.260697,30.520188,7516.234360
6,2014,269.300317,137.053626,230.340534,480.557804,413.096700,190.215533,610.147141,232.741071,233.813861,...,254.492145,287.032682,458.352797,137.277769,128.276751,178.276980,229.407034,257.170089,46.513653,6507.972511
7,2015,179.806633,65.815077,244.293125,360.040058,426.466306,174.591565,476.673273,205.546281,231.524550,...,296.522205,194.464888,611.455069,80.151458,55.495723,201.754248,183.463176,147.631791,25.130203,6087.017228
8,2016,182.642363,68.054379,166.405646,394.086380,284.168869,161.349181,438.020619,170.827404,212.622927,...,229.182173,169.606359,440.072514,55.522815,69.733465,236.932385,159.933790,248.397689,15.309218,5661.862857
9,2017,246.627437,118.673963,173.731005,680.170603,265.751651,158.574732,344.754504,170.770580,174.906352,...,267.489834,263.370255,359.154378,144.192782,66.247646,283.461953,220.935837,74.443757,31.661236,5919.158214


## TABLA A VALORES REALES

In [15]:
directorio='C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs'
os.chdir(directorio)
deflactor=pd.read_excel('DEFLACTOR.xlsx')
def_1=deflactor[18:].reset_index(drop=True)
df_real=df.copy()
#print(def_1)
#print(df.dtypes)
#deflactor['DEFLACTOR'] = pd.to_numeric(deflactor['DEFLACTOR'], errors='coerce')
df_real.iloc[:, 1:] = df_real.iloc[:, 1:].div(def_1['DEFLACTOR'], axis=0)
df_real.iloc[:, 1:] = df_real.iloc[:, 1:] * 100
df_real

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,Huánuco,...,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Lima,Callao,Municipalidad Metropolitana de Lima,Total
0,2008,106.037190,134.887936,78.880269,195.579176,98.498381,69.880485,242.603493,128.091132,84.034162,...,145.792459,90.208299,120.298865,92.376951,53.000022,83.507835,76.948627,66.452500,44.054468,2726.613587
1,2009,141.919570,281.549606,68.861237,295.603728,121.932169,202.797517,294.022805,132.829226,70.973160,...,195.113098,225.436950,213.931925,136.847419,78.000372,145.775697,108.481042,90.677022,133.978892,3973.317558
2,2010,106.491421,490.349338,128.822362,288.596299,166.260143,249.854405,255.824314,165.044468,99.216788,...,299.278067,171.468594,201.494929,114.104741,99.596081,130.838777,135.910579,119.336985,247.546639,4570.610801
3,2011,122.686349,558.753819,97.050792,295.638021,119.928212,396.266244,209.843173,145.674056,166.913305,...,168.499089,100.183378,225.290926,62.859670,117.038897,118.195520,86.441225,102.891565,9.565254,4141.972232
4,2012,189.390040,589.437555,157.580979,313.599630,221.161193,325.656779,514.507681,199.918265,254.659679,...,304.701629,190.706722,302.246780,153.301704,181.628084,109.692192,180.985311,308.549822,65.595269,5856.100340
5,2013,205.908641,300.864523,194.714066,476.793555,300.041402,225.424597,578.635078,187.520684,205.606953,...,350.379112,262.984728,340.053868,165.979092,138.140680,170.593221,210.417323,279.719566,25.388319,6252.404258
6,2014,217.541336,110.712194,186.069545,388.195558,333.700343,153.656489,492.878084,188.008704,188.875306,...,205.579265,231.865576,370.258309,110.893257,103.622216,144.012502,185.315462,207.742514,37.573822,5257.153243
7,2015,141.241554,51.699004,191.896928,282.818361,334.997451,137.145018,374.435985,161.460540,181.866968,...,232.923871,152.755893,480.309666,62.960505,43.592953,158.481826,144.113838,115.967599,19.740256,4781.468605
8,2016,138.218985,51.501782,125.931460,298.234312,215.051601,122.104859,331.482600,129.277732,160.907495,...,173.439101,128.353676,333.035420,42.018221,52.772471,179.304260,121.033728,187.980904,11.585618,4284.750393
9,2017,179.571817,86.407657,126.495221,495.238779,193.496341,115.459792,251.019081,124.339707,127.351003,...,194.761929,191.762424,261.503768,104.988156,48.235550,206.391383,160.865515,54.203218,23.052851,4309.796212


## TABLA COMO PORCENTAJE DEL PBI


In [16]:
df_por=df_real.copy()
#print(def_1)
df_por.iloc[:, 1:] = df_por.iloc[:, 1:].div(def_1['PBI_real'], axis=0)
#df_real.iloc[:, 1:] = df_real.iloc[:, 1:] * 100
directorio='C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs/inversion bruta/gob_regional'
os.chdir(directorio)
df_por.to_excel('porcentaje_inv_bruta_gregionales.xlsx', index=False)
df_real.to_excel('ureales_inv_bruta_gregionales.xlsx',index=False)
df.to_excel('unominales_inv_bruta_gregionales.xlsx',index=False)
df_por

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,Huánuco,...,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Lima,Callao,Municipalidad Metropolitana de Lima,Total
0,2008,0.000304,0.000387,0.000226,0.000561,0.000282,0.000200,0.000695,0.000367,0.000241,...,0.000418,0.000259,0.000345,0.000265,0.000152,0.000239,0.000221,0.000190,0.000126,0.007816
1,2009,0.000402,0.000798,0.000195,0.000838,0.000346,0.000575,0.000834,0.000377,0.000201,...,0.000553,0.000639,0.000607,0.000388,0.000221,0.000413,0.000308,0.000257,0.000380,0.011266
2,2010,0.000279,0.001283,0.000337,0.000755,0.000435,0.000654,0.000670,0.000432,0.000260,...,0.000783,0.000449,0.000527,0.000299,0.000261,0.000342,0.000356,0.000312,0.000648,0.011962
3,2011,0.000302,0.001375,0.000239,0.000728,0.000295,0.000975,0.000517,0.000359,0.000411,...,0.000415,0.000247,0.000555,0.000155,0.000288,0.000291,0.000213,0.000253,0.000024,0.010195
4,2012,0.000439,0.001367,0.000365,0.000727,0.000513,0.000755,0.001193,0.000464,0.000591,...,0.000707,0.000442,0.000701,0.000356,0.000421,0.000254,0.000420,0.000716,0.000152,0.013581
5,2013,0.000451,0.000659,0.000427,0.001045,0.000657,0.000494,0.001268,0.000411,0.000450,...,0.000768,0.000576,0.000745,0.000364,0.000303,0.000374,0.000461,0.000613,0.000056,0.013698
6,2014,0.000466,0.000237,0.000398,0.000831,0.000714,0.000329,0.001055,0.000402,0.000404,...,0.000440,0.000496,0.000792,0.000237,0.000222,0.000308,0.000397,0.000445,0.000080,0.011250
7,2015,0.000293,0.000107,0.000398,0.000586,0.000694,0.000284,0.000776,0.000335,0.000377,...,0.000483,0.000317,0.000995,0.000130,0.000090,0.000328,0.000299,0.000240,0.000041,0.009910
8,2016,0.000276,0.000103,0.000251,0.000595,0.000429,0.000243,0.000661,0.000258,0.000321,...,0.000346,0.000256,0.000664,0.000084,0.000105,0.000357,0.000241,0.000375,0.000023,0.008542
9,2017,0.000349,0.000168,0.000246,0.000963,0.000376,0.000225,0.000488,0.000242,0.000248,...,0.000379,0.000373,0.000509,0.000204,0.000094,0.000401,0.000313,0.000105,0.000045,0.008381
